In [2]:
import numpy as np
import tensorflow as tf
import keras
from keras import Model, activations, layers, losses, optimizers, callbacks, regularizers
from keras.layers import Dense, Convolution1D, Dropout, BatchNormalization, concatenate, TimeDistributed, Layer, Input
import matplotlib.pyplot as plt
from ss_functions import *
from ss_pred_classes import *

In [3]:
x_data, y_data = get_data('list.txt')
x_train, y_train = x_data[101:], y_data[101:]
fff, y_data = get_data('list.txt', encode_y=False)
x_test, y_test = x_data[:101], y_data[:101]

In [2]:
x_data, y_data = get_data1('list.txt',padding=False)
x_train3, y_train3 = x_data[101:], y_data[101:]
fff, y_data = get_data1('list.txt', encode_y=False,padding=False)
x_test2, y_test2 = x_data[:101], y_data[:101]

In [ ]:
x_data1, y_data1 = get_data2('list.txt')
x_train1, x_train2 = np.array(x_data1[0][101:]), np.array(x_data1[1][101:])
x_test1, x_test2 = np.array(x_data1[0][:101]), np.array(x_data1[1][:101])

In [9]:
def get_model0():
    inputs = Input((None, 41))
    X = inputs
    # X = layers.Masking()(X)
    X = DeepInception_block()(X)
    X = DeepInception_block()(X)
    # X = DeepInception_block()(X)

    X = Convolution1D(100, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
    X = TimeDistributed(Dense(256, activation='relu'))(X)
    X = Dropout(0.4)(X)

    Y = TimeDistributed(Dense(3, activation='softmax'))(X)
    model = Model(inputs=inputs, outputs=Y)
    return model

In [1]:
model = get_model0()

opt = optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', # try siome: "categorical_focal_crossentropy, adam, sparse_categorical_crossentropy
              optimizer=opt,
              metrics=['accuracy', 'mae', truncated_accuracy])
model.summary()

NameError: name 'get_model0' is not defined

In [13]:
stop_monitor_loss = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=8,
    mode='min',
    verbose=1,
    restore_best_weights=True
)

checkpoint = callbacks.ModelCheckpoint(
    './ss_pred_model0.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

lr_fun = callbacks.LearningRateScheduler(
    learn_decay
)

with tf.device('/GPU:1'):
    history = model.fit(x_train, y_train,
                        epochs=100,
                        batch_size=24,
                        validation_split=0.1,
                        callbacks=[stop_monitor_loss, checkpoint])
    
# 3rd block - negligeable improvement

Epoch 1/100
42/42 [==============================] - ETA: 0s - loss: 2.0844 - accuracy: 0.8403 - mae: 0.3536 - truncated_accuracy: 0.4143
Epoch 1: val_accuracy improved from -inf to 0.86390, saving model to .\ss_pred_model0.keras
42/42 [==============================] - 23s 544ms/step - loss: 2.0844 - accuracy: 0.8403 - mae: 0.3536 - truncated_accuracy: 0.4143 - val_loss: 1.6904 - val_accuracy: 0.8639 - val_mae: 0.3509 - val_truncated_accuracy: 0.4132
Epoch 2/100
42/42 [==============================] - ETA: 0s - loss: 1.2846 - accuracy: 0.8133 - mae: 0.3506 - truncated_accuracy: 0.4239
Epoch 2: val_accuracy did not improve from 0.86390
42/42 [==============================] - 16s 374ms/step - loss: 1.2846 - accuracy: 0.8133 - mae: 0.3506 - truncated_accuracy: 0.4239 - val_loss: 1.0610 - val_accuracy: 0.8639 - val_mae: 0.3509 - val_truncated_accuracy: 0.4132
Epoch 3/100
42/42 [==============================] - ETA: 0s - loss: 0.7895 - accuracy: 0.9032 - mae: 0.3394 - truncated_accuracy

In [13]:
test_model = keras.models.load_model('ss_pred_model0.keras', 
                                     custom_objects={
                                         'inception_conv': inception_conv,
                                         'InceptionNet_paper': InceptionNet_paper,
                                         'DeepInception_block': DeepInception_block,
                                         'truncated_accuracy': truncated_accuracy
                                     })

In [14]:
ss_map = {'C': 0, 'H': 1, 'E': 2}
from_aa = {0: 'C', 1: 'H', 2: 'E'}
predictions_hot = test_model.predict(x_test)
predictions = []
for prediction in predictions_hot:
    dssp = ''
    for i in prediction:
        dssp += from_aa[np.argmax(i)]
    predictions.append(dssp)

total = 0
TP = 0
for prediction, truth in zip(predictions, y_test):
    for i, ss in enumerate(truth):
        total +=1
        if ss==prediction[i]:
            TP+=1

accuracy = TP/total
print(accuracy)

4/4 [==============================] - 5s 251ms/step
0.47987097574592147


In [ ]:
block1 = DeepInception_block()
block2 = DeepInception_block()
block3 = DeepInception_block()

inputs = layers.Input((800,41))
X = block1(inputs)
X = block2(X)
X1 = Convolution1D(100, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
X1 = TimeDistributed(Dense(256, activation='relu'))(X1)
X1 = Dropout(0.5)(X1)

Y2 = TimeDistributed(Dense(3, activation='softmax'))(X1)
X = block3(X)

# input1 = layers.Input((800,21))
# input2 = layers.Input((800,20))
# X1 = block1(input1)
# X2 = block2(input2)

# X = layers.concatenate([X1,X2])
# X = DeepInception_block()(X)

X = Convolution1D(100, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
X = TimeDistributed(Dense(256, activation='relu'))(X)
X = Dropout(0.5)(X)

Y1 = TimeDistributed(Dense(3, activation='softmax'))(X)

model = Model(inputs=inputs, outputs=[Y1,Y2])

opt = optimizers.Adam(learning_rate=0.005)
model.compile(loss='categorical_crossentropy', # try siome: "categorical_focal_crossentropy, adam, sparse_categorical_crossentropy
              optimizer=opt,
              metrics=[truncated_accuracy])

stop_monitor_loss = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=8,
    mode='min',
    verbose=1,
    restore_best_weights=True
)

checkpoint = callbacks.ModelCheckpoint(
    './ss_pred_modeltest9.keras',
    monitor='val_truncated_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)


input:
1. one hot encoded sequence
2. PSSM

Model:
1D convolutional neural network

output:
multiclass classification - dense layer with relu activaiton - 3?

validation metric - accuray + model specific measures

soruces:
https://www.csbj.org/article/S2001-0370(22)00506-2/fulltext
